In [ ]:
using BenchmarkTools
using DataFrames
using DelimitedFiles
using CSV
using XLSX
using Downloads

In [ ]:
P = Downloads.download("https://raw.githubusercontent.com/nassarhuda/easy_data/master/programming_languages.csv",
    "programming_languages.csv")

In [ ]:
P,H = readdlm("programming_languages.csv",',';header=true);

In [ ]:
H

In [ ]:
#read CSV Files
C = CSV.read("programming_languages.csv", DataFrame);

In [ ]:
@show typeof(C)
C[1:10,:]
#C[!,:year]

In [ ]:
@show typeof(P)
P[1:10,:]

In [ ]:
names(C)

In [ ]:
names(C)
C.year
C.language
describe(C)

In [ ]:
#Benchmark Tools

@btime P,H = readdlm("programming_languages.csv",',';header=true);
@btime C = CSV.read("programming_languages.csv", DataFrame);

In [ ]:
CSV.write("programminglanguages_CSV.csv", DataFrame(P, :auto))

In [ ]:
#read XLSX files

# j = XLSX.readdata('File Name.xlsx','Sheet Name','Cell Range')

T = XLSX.readdata("data/zillow_data_download_april2020.xlsx","Sale_counts_city","A1:F9")

In [ ]:
#read Table on XLSX

G = XLSX.readtable("data/zillow_data_download_april2020.xlsx","Sale_counts_city");

In [ ]:
G[1]

In [ ]:
G[1][1][1:10]

In [ ]:
G[2][1:10]

In [ ]:
D = DataFrame(G...) # equivalent to DataFrame(G[1],G[2])

In [ ]:
foods = ["apple", "cucumber", "tomato", "banana"]
calories = [105,47,22,105]
prices = [0.85,1.6,0.8,0.6,]
dataframe_calories = DataFrame(item=foods,calories=calories)
dataframe_prices = DataFrame(item=foods,price=prices)

In [ ]:
DF = innerjoin(dataframe_calories,dataframe_prices,on=:item)

In [ ]:
DataFrame(T, :auto)

In [ ]:
# if you already have a dataframe: 
# XLSX.writetable("filename.xlsx", collect(DataFrames.eachcol(df)), DataFrames.names(df))
XLSX.writetable("writefile_using_XLSX.xlsx",G[1],G[2])

In [ ]:
#read JLD(Julia Language) data

using JLD
jld_data = JLD.load("data/mytempdata.jld")
save("mywrite.jld", "A", jld_data)

In [ ]:
#read NPZ data

using NPZ
npz_data = npzread("data/mytempdata.npz")
npzwrite("mywrite.npz", npz_data)

In [ ]:
#read MATLAB Data

using MAT
Matlab_data = matread("data/mytempdata.mat")
matwrite("mywrite.mat",Matlab_data)

In [ ]:
#show all Files

@show typeof(jld_data)
@show typeof(npz_data)
@show typeof(Matlab_data)
;

In [ ]:
Matlab_data

In [ ]:
P

In [ ]:
# Q1: Which year was was a given language invented?
function year_created(P,language::String)
    loc = findfirst(P[:,2] .== language)
    return P[loc,1]
end
year_created(P,"Julia")

In [ ]:
year_created(P,"W") 

In [ ]:
function year_created_handle_error(P,language::String)
    loc = findfirst(P[:,2] .== language)
    !isnothing(loc) && return P[loc,1]
    error("Error: Language not found.")
end
year_created_handle_error(P,"W")

In [ ]:
# Q2: How many languages were created in a given year?

function how_many_per_year(P,year::Int64)
    year_count = length(findall(P[:,1].==year))
    return year_count
end
how_many_per_year(P,2011)

In [ ]:
P_df = C #DataFrame(year = P[:,1], language = P[:,2]) # or DataFrame(P)

In [ ]:
# Q1: Which year was was a given language invented?
# it's a little more intuitive and you don't need to remember the column ids
function year_created(P_df,language::String)
    loc = findfirst(P_df.language .== language)
    return P_df.year[loc]
end
year_created(P_df,"Julia")

In [ ]:
year_created(P_df,"W")

In [ ]:
function year_created_handle_error(P_df,language::String)
    loc = findfirst(P_df.language .== language)
    !isnothing(loc) && return P_df.year[loc]
    error("Error: Language not found.")
end
year_created_handle_error(P_df,"W")

In [ ]:
# Q2: How many languages were created in a given year?
function how_many_per_year(P_df,year::Int64)
    year_count = length(findall(P_df.year.==year))
    return year_count
end
how_many_per_year(P_df,2011)

In [ ]:
# A quick example to show how to build a dictionary
Dict([("A", 1), ("B", 2),(1,[1,2])])

In [ ]:
P_dictionary = Dict{Integer,Vector{String}}()

In [ ]:
#P_dictionary[67] = ["julia","programming"]

In [ ]:
#this is not gonna work.
#P_dictionary["julia"] = 7

In [ ]:
dict = Dict{Integer,Vector{String}}()
for i = 1:size(P,1)
    year,lang = P[i,:]
    if year in keys(dict)
        dict[year] = push!(dict[year],lang) 
        # note that push! is not our favorite thing to do in Julia, 
        # but we're focusing on correctness rather than speed here
    else
        dict[year] = [lang]
    end
end

In [ ]:

# Though a smarter way to do this is:
curyear = P_df.year[1]
P_dictionary[curyear] = [P_df.language[1]]
for (i,nextyear) in enumerate(P_df.year[2:end])
    if nextyear == curyear
        #same key
        P_dictionary[curyear] = push!(P_dictionary[curyear],P_df.language[i+1])
        # note that push! is not our favorite thing to do in Julia, 
        # but we're focusing on correctness rather than speed here
    else
        curyear = nextyear
        P_dictionary[curyear] = [P_df.language[i+1]]
    end
end

In [ ]:
length(keys(P_dictionary))

In [ ]:
length(unique(P[:,1]))

In [ ]:

# Q1: Which year was was a given language invented?
# now instead of looking in one long vector, we will look in many small vectors
function year_created(P_dictionary,language::String)
    keys_vec = collect(keys(P_dictionary))
    lookup = map(keyid -> findfirst(P_dictionary[keyid].==language),keys_vec)
    # now the lookup vector has `nothing` or a numeric value. We want to find the index of the numeric value.
    return keys_vec[findfirst((!isnothing).(lookup))]
end
year_created(P_dictionary,"Julia")

In [ ]:
# Q2: How many languages were created in a given year?
how_many_per_year(P_dictionary,year::Int64) = length(P_dictionary[year])
how_many_per_year(P_dictionary,2011)

In [ ]:
# assume there were missing values in our dataframe
P[1,1] = missing
P_df = DataFrame(year = P[:,1], language = P[:,2])

In [ ]:
dropmissing(P_df)

# Finally...
### After finishing this notebook, you should be able to:

#### [ ] dowload a data file from the web given a url
#### [ ] load data from a file from a text file via DelimitedFiles or CSV
#### [ ] write your data to a text file or csv file
#### [ ] load data from file types xlsx, jld, npz, mat, rda
#### [ ] write your data to an xlsx file, jld, npz, mat, rda
#### [ ] store data in a 2D array (Matrix), or DataFrame or Dict
#### [ ] write functions to perform basic lookups on Matrix, DataFrame, and Dict types
#### [ ] use some of the basic functions on DataFrames such as: dropmissing, describe, by, and join